In [1]:
!pwd

/mystuff/notebooks


In [ ]:
!ls

In [3]:
!ls ..

data  jupyter_start.sh	notebooks


In [4]:
%cd ../data

/mystuff/data


In [5]:
!pwd

/mystuff/data


In [ ]:
!ls

In [ ]:
!nvidia-smi

In [8]:
!pip install nvidia-ml-py

In [9]:
from pynvml import nvmlInit, nvmlDeviceGetMemoryInfo, nvmlDeviceGetHandleByIndex, nvmlDeviceGetCount

In [10]:
nvmlInit()
handle = nvmlDeviceGetHandleByIndex(0)
device_count = nvmlDeviceGetCount()


In [11]:
def print_nvmi_meminfo():
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"device count: {device_count}")
    print(f"Total memory: {info.total}")
    print(f"Free memory:, {info.free}")
    print(f"Used memory: {info.used}")


In [12]:
print_nvmi_meminfo()

device count: 1
Total memory: 25769803776
Free memory:, 25372917760
Used memory: 396886016


In [ ]:
!python -m torch.utils.collect_env 

In [14]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Jun__8_16:49:14_PDT_2022
Cuda compilation tools, release 11.7, V11.7.99
Build cuda_11.7.r11.7/compiler.31442593_0


In [ ]:
!python -m bitsandbytes

In [16]:
!ls 

Qwen-7B					 minotaur-15b	starcoder
WizardLM-1.0-Uncensored-Llama2-13B-GPTQ  processed	vicuna-13b-v1.5-16k
WizardLM-33B-V1.0-Uncensored-GPTQ	 starchat-beta


In [17]:
%cd starchat-beta/human-eval

/mystuff/data/starchat-beta/human-eval


In [ ]:
!ls

In [19]:
#!pip install -e human-eval
!pip install -e .

Obtaining file:///mystuff/data/starchat-beta/human-eval
  Preparing metadata (setup.py) ... done
  Attempting uninstall: human-eval
    Found existing installation: human-eval 1.0
    Uninstalling human-eval-1.0:
      Successfully uninstalled human-eval-1.0
  Running setup.py develop for human-eval


In [20]:
from human_eval.data import write_jsonl, read_problems
from human_eval import execution

problems = read_problems()

In [21]:
system_prompt="you are a smart programming assistant. provide a python3 implementation of the following function:\n"


In [22]:
%cd ../..

/mystuff/data


In [23]:
import time

int(time.time())


1691388201

In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("starchat-beta", local_files_only=True)
# to save memory consider using fp16 or bf16 by specifying torch_dtype=torch.float16 for example
model = AutoModelForCausalLM.from_pretrained("starchat-beta", local_files_only=True, load_in_4bit=True)
#model = AutoModelForCausalLM.from_pretrained("starchat-beta", local_files_only=True, load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of GPTBigCodeForCausalLM were not initialized from the model checkpoint at starchat-beta and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
import time

int(time.time())


1691388265

In [26]:
%%time
import torch
from transformers import pipeline

pipe = pipeline("text-generation", 
                model=model,
                tokenizer=tokenizer, 
                torch_dtype=torch.bfloat16, 
                device_map="auto")

# We use a variant of ChatML to format each message
#prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
prompt_template = "<|system|>{system_prompt}\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


CPU times: user 78.1 ms, sys: 2.74 ms, total: 80.8 ms
Wall time: 80.1 ms


In [27]:
import time

int(time.time())


1691388269

In [ ]:
from tqdm.notebook import tqdm
import time

user_prompts = []
canonical_solutions = []
problem_tests = []
problem_keys = []
answers = []
times = []
system_prompt="you are a smart programming assistant. provide a python3 implementation of the following function:"

for idx, problem_id in tqdm(enumerate(problems)):
    problem = problems[problem_id]
    problem_keys.append(problem_id)
    user_prompt=problem["prompt"]
    canonical_solution=problem["canonical_solution"]
    problem_test=problem["test"]
    problem_tests.append(problem_test)
    prompt = prompt_template.format(system_prompt=system_prompt, query=user_prompt)
    start_time = time.time()
    outputs = pipe(prompt,
                   max_new_tokens=1024,
                   do_sample=False,
#                   temperature=0.0, #0 is not allowed, so do_sample=False
#                   temperature=0.2,
#                   top_k=50,
#                   top_p=0.95,
                   eos_token_id=49155)
    answer = outputs[0]["generated_text"]
    end_time = time.time()
    diff_time = end_time - start_time
    answers.append(answer)
    times.append(diff_time)
#    if idx > 10:
#        break


In [29]:
import time

int(time.time())


1691390684

In [30]:
import pandas as pd

df_generation = pd.DataFrame()
df_generation["prompt"] = user_prompts
df_generation["canonical"] = canonical_solution
df_generation["test"] = problem_tests
df_generation["answer"] = answers
df_generation["problem_id"] = problem_keys
df_generation["time"] = times


In [31]:
df_generation

,prompt,canonical,test,answer,problem_id,time
0,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/0,22.788677
1,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/1,8.509821
2,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/2,8.839592
3,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/3,5.262252
4,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/4,10.686575
...,...,...,...,...,...,...
159,NaN,NaN,def check(candidate):\n\n # Check some simp...,<|system|>you are a smart programming assistan...,HumanEval/159,2.690794
160,NaN,NaN,def check(candidate):\n\n # Check some simp...,<|system|>you are a smart programming assistan...,HumanEval/160,17.400587
161,NaN,NaN,def check(candidate):\n\n # Check some simp...,<|system|>you are a smart programming assistan...,HumanEval/161,5.818320
162,NaN,NaN,def check(candidate):\n\n # Check some simp...,<|system|>you are a smart programming assistan...,HumanEval/162,3.498783


In [32]:
import re

def slice_until_word(s, word):
    start_index = s.find(word)
    if start_index == -1:
        # word not found in string
        return s
    end_index = start_index + len(word)
    return s[end_index:]


def slice_fundction_definition(completion, words):
    completion = slice_until_word(completion, "<|assistant|>")

    lines = completion.split('\n')  # split the string into lines
    result = []
    capture = False

    for line in lines:
        if line:
            if len(line.strip()) > 0:
                first_word = line.split()[0]
                if first_word in words:
                    capture = True
                elif capture and (line[0].isalpha() or line[0] == '`'):
                    capture = False
        if capture:
#            print(line)
            result.append(line)

    return '\n'.join(result)

def extract_python_code(input_string):
#    if "```python" in input_string:    
#        python_code = re.findall("```python(.*?)```", input_string, re.DOTALL)
#        answers = [code.strip() for code in python_code]
#        return answers[0]
    return slice_fundction_definition(input_string, ["from", "import", "def"])


In [33]:
def execute_python_code(code, function_name, *args):
#    code = "import math\n"+code
    exec(code)
    result = locals()[function_name](*args)
    return result


In [34]:
import time

int(time.time())


1691390694

In [ ]:
exec_results = []
exec_times = []
extracted_codes = []
for idx, answer in tqdm(enumerate(answers)):
    key = problem_keys[idx]
    problem = problems[key]
    extracted_code = extract_python_code(answer)
    code_to_check = extracted_code
    code_to_check = "import math\n"+extracted_code
    extracted_codes.append(code_to_check)
    #params: problem, generated code (completion), timeout, completion_id
    start_time = time.time()
    result = execution.check_correctness(problem, code_to_check, 10, 1)
    end_time = time.time()
    time_diff = end_time - start_time
    exec_results.append(result)
    exec_times.append(time_diff)

In [36]:
import time

int(time.time())


1691390759

In [37]:
pass_fail = [1 if er["passed"] else 0 for er in exec_results ]
task_ids_2 = [er["task_id"] for er in exec_results]

#try 1 without python version definition gave 46 correct
#try 2 with python3 definition gave 46 correct also
#try 3 after prepending try 2 code with "import math" = 48 passed
#try 4 using no quantization, import math = 49 passed
#5 8bit, temp 0.2, 44, math, 256
#6 8bit, temp 0, 46, math, 256
#7 8bit, temp 0, 44, no math, 256
#8 4bit, temp 0, 42, no math, 256
#9 4bit, temp 0, 44, math, 256
#10 4 bit, temp 0, 47, math, 1024
#11 4 bit, temp 0, 45, nomath, 1024
#12 8 bit, temp 0, 47, nomath, 1024
#13 8 bit, temp 0, 49, math, 1024
#14 CPU: 
sum(pass_fail)

47

In [38]:
import pandas as pd

df_passfail = pd.DataFrame()
df_passfail["pass"] = pass_fail
df_passfail["problem_id"] = task_ids_2
df_passfail["exec_time"] = exec_times
df_passfail

,pass,problem_id,exec_time
0,1,HumanEval/0,0.071141
1,0,HumanEval/1,0.059336
2,1,HumanEval/2,0.059078
3,1,HumanEval/3,0.059538
4,0,HumanEval/4,0.059678
...,...,...,...
159,0,HumanEval/159,0.068854
160,0,HumanEval/160,0.068247
161,0,HumanEval/161,0.068586
162,1,HumanEval/162,0.068269


In [39]:
merged_df = df_generation.merge(df_passfail, on='problem_id')

In [40]:
!ls results

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ls: cannot access 'results': No such file or directory


In [44]:
%cd starchat-beta

/mystuff/data/starchat-beta


In [45]:
merged_old_df = pd.read_csv("results/gpu_1024_4b_temp00_math_stats.csv")

In [46]:
merged_df.head()

,prompt,canonical,test,answer,problem_id,time,pass,exec_time
0,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/0,22.788677,1,0.071141
1,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/1,8.509821,0,0.059336
2,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/2,8.839592,1,0.059078
3,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/3,5.262252,1,0.059538
4,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/4,10.686575,0,0.059678


In [47]:
merged_old_df.head()

,Unnamed: 0,prompt,canonical,test,answer,problem_id,time,pass,exec_time
0,0,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/0,60.665479,1,0.065942
1,1,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/1,48.595213,0,0.059510
2,2,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/2,48.404578,1,0.059203
3,3,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/3,28.898859,1,0.059048
4,4,NaN,NaN,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",<|system|>you are a smart programming assistan...,HumanEval/4,55.860152,0,0.059134


In [48]:
merged_df.to_csv("results/gpu_1024_4b_v3.csv")

In [54]:
sum(merged_df["pass"])

47

In [56]:
sum(merged_old_df["pass"])

47

In [50]:
def print_completion_evaluation(idx):
    print(extracted_codes[idx])
    print(exec_results[idx])

In [ ]:
for x in range(len(problems)):
    print_completion_evaluation(x)
    print("-"*100)
